In [1]:
import pandas as pd
import numpy as np
from pandasql import sqldf

q = lambda q: sqldf(q, globals())

data = pd.read_csv("source_data/datasets_26532_33799_super_hero_powers.csv")
data.columns= data.columns.str.lower().str.replace("[^a-zA-Z0-9]+","_")
numerical_data = data[data.columns.values[range(1,len(data.columns.values))]]

superman = q("select * from data where hero_names=\"Superman\"")
superman_n = superman[data.columns.values[range(1,len(data.columns.values))]].values;

for column in numerical_data.columns.values:
    numerical_data[column] = numerical_data[column].astype(int)
    


training = numerical_data.values
training = training.astype(float)
training.shape
training

training == np.repeat(superman_n, (1, training.shape[1]))

/tmp/ipykernel_45/3729995485.py:8: FutureWarning: The default value of regex will change from True to False in a future version.
  data.columns= data.columns.str.lower().str.replace("[^a-zA-Z0-9]+","_")
/tmp/ipykernel_45/3729995485.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  numerical_data[column] = numerical_data[column].astype(int)


ValueError: operands could not be broadcast together with shape (167,) (2,)

In [2]:
from keras.layers import Input,Conv2D,MaxPooling2D,UpSampling2D,Dense, LeakyReLU
from keras.layers import Dropout
from keras.models import Model
from keras.optimizers import rmsprop_v2 as RMSProp

latent_size = 2;
middle_units = 45;

input = Input(shape=(training.shape[1],));
enc = Dropout(rate=0.2)(input);
enc = LeakyReLU()(enc);
enc = Dense(units=middle_units)(enc)
enc = Dense(units=middle_units)(enc)
enc = Dense(units=latent_size)(enc);

encoder = Model(input, enc)

dec_input = Input(shape=(latent_size,));
dec = Dense(units=middle_units)(dec_input);
dec = Dense(units=middle_units)(dec);
dec = LeakyReLU()(dec);
dec = Dense(units=(training.shape[1]))(dec)

decoder = Model(dec_input, dec)



2021-11-10 19:16:18.406344: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-11-10 19:16:18.406367: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2021-11-10 19:16:20.007261: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2021-11-10 19:16:20.007294: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2021-11-10 19:16:20.007322: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (c38eccc005f5): /proc/driver/nvidia/version does not exist
2021-11-10 19:16:20.007731: I tensorflow/core/platform/cpu_featu

In [3]:
aen_input = Input(shape=(training.shape[1]))
aen_encoder_output = encoder(aen_input);
aen_decoder_output = decoder(aen_encoder_output);

aen = Model(aen_input, aen_decoder_output);
aen.compile(optimizer="rmsprop", loss="mean_absolute_error")

In [4]:
aen.fit(numerical_data, numerical_data, epochs=2000, verbose=False)
e = pd.DataFrame(encoder.predict(numerical_data),columns=["E1","E2"])
reconstructed = pd.DataFrame(decoder.predict(e) > 0.5, columns = numerical_data.columns.values)
for column in reconstructed.columns.values:
    reconstructed[column] = reconstructed[column].astype(int)

tmp = numerical_data.values != reconstructed.values
np.sum(np.abs(tmp),axis=1)



2021-11-10 19:16:46.259322: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


array([ 0,  5, 11,  1,  6, 13,  9,  3, 11,  1,  5,  5,  3, 13, 23,  1, 11,
        1, 39,  1,  1, 14,  0,  5,  1,  5,  1,  2,  7, 21,  4, 12, 17,  2,
        9, 17,  5,  5,  2, 13,  1,  2,  2,  7,  8,  4,  1,  4,  1,  1, 10,
        6,  3, 11,  3,  3,  6,  1,  4,  4,  2, 12,  3,  0,  3,  3,  8, 11,
        2,  5, 18,  7,  1,  1,  2,  1,  4,  2,  3,  3,  2,  8,  7, 18, 13,
        5, 12,  8,  3,  3,  9,  1,  7,  3, 14,  7,  4,  3,  3,  2,  2,  7,
        3,  1,  2,  1,  1,  8,  7,  8,  2,  1, 11,  3,  6,  2,  6, 16,  1,
        1,  9,  3,  1,  4,  5,  4,  9,  2,  2, 25,  7,  3,  5, 10, 25,  5,
        2,  2,  3, 26,  5,  4, 24,  8,  9,  3,  5,  3,  1,  4,  1,  3,  3,
        2, 13, 10,  1,  4,  2,  3,  5,  2,  5,  1,  5,  0,  4,  2,  6,  3,
        5,  8, 15,  7,  1,  3,  1,  8,  8,  1, 22,  5, 12, 24,  2, 13, 13,
       12,  9,  1,  5,  6,  1,  4,  4,  2,  1, 12,  4, 21,  6,  2, 20,  5,
        4,  2, 13,  3, 11, 27,  8,  2,  5,  3,  7,  1,  6, 12, 12,  3,  8,
        8,  9,  8, 12,  1

In [27]:
import bokeh as bk
from bokeh.plotting import figure
from bokeh.models import ColumnDataSource;
from ipywidgets import interact
from bokeh.io import output_notebook, show
from bokeh.models.tools import HoverTool

output_notebook();

data['E1'] = e['E1'];
data['E2'] = e['E2'];

source = ColumnDataSource(data);

tools="box_zoom, reset, box_select"

f = figure(plot_width=400, plot_height=400, tools = tools);
h = HoverTool();
h.tooltips = [('hero','@hero_names')]
f.add_tools(h)
f.circle('E1','E2',source=source);

show(f)

Loading BokehJS ...